<a href="https://colab.research.google.com/github/dudwn98/iipl_topic_modeling/blob/main/LDA_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import argparse
import pandas as pd
import re
import konlpy

In [50]:
def preprocessing(news):
    #개행 제거
    news = re.sub('\\\\n', '', news)
    #이메일 제거
    news = re.sub(r'\b[\w\+]+@[\w]+.[\w]+.[\w]+.[\w]+\b', ' ', news)
    #특수문자, 자음 제거
    news = re.sub(r'[^ㄱ-ㅣ가-힣A-Za-z0-9]',' ', news)
    #중복 공백 제거
    news = re.sub(' +', ' ', news)
    return news

In [90]:
def remove_stopwords(text):
    okt = konlpy.tag.Okt()
    tokens = okt.nouns(text)

    stops = ['기자', '뉴스', '경향', '뉴시스', '무단 배포', '금지', '무단',
             '위해', '보기', '배포', '비롯', '통해', '통한', '마련', '다른',
             '가지', '여개', '개', '또', '평']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [91]:
path='/content/drive/MyDrive/크롤링 데이터/2018/2017서울포커스.csv'

file = pd.read_csv(path, encoding='cp949')
df = pd.DataFrame({'doc' : file.기사제목 + file.기사내용})

df.loc[:, 'doc'] = df['doc'].apply(preprocessing)
df.loc[:, 'doc'] = df['doc'].apply(remove_stopwords)
print(df)
## 텍스트 데이터를 단어 빈도수에 기반해 벡터화시키기
# max_df = 토큰이 나타날 최대 문서 개수 -> 소수점이면 비중
# max_features = corpus 중 빈도수가 가장 높은 순으로 해당 개수만큼만 뽑아냄
# min_df = 토큰이 나타날 최소 문서 개수 -> 정수면 횟수
# ngram_range = (min_n, max_n) 튜플, 단어장 생성에 사용할 토큰의 크기를 결정
count_vect = CountVectorizer(max_df=0.95, max_features=1000,
                            min_df=2, ngram_range=(1,1))
# feature 리스케일링 해서 feature 평균이 0 분산이 1 되게 만들어줌
#fit_transfrom은 train dataset에서만 사용
ftr_vect = count_vect.fit_transform(df.doc)

# LDA 클래스 사용해 피쳐 벡터화시킨 거 토픽모델링
# n_components = 토픽개수
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(ftr_vect)

# 각 토픽
print(lda.components_.shape)
# 단어들을 벡터화한 feature
print(lda.components_)

                                                 doc
0  새해 나들이 멀리 서울 공연 전시 세종 문화 회관 신년 음악회 서울시 제공 신년 음...
1  상계 시가지 준공 주년 서울 김성수 상계동 철거 지역 사진 서울 시립 북 서울 미술...
2  황금 띠해 달 서울시 곳곳 문화 예술 프로그램 성 위 사진 특정 사실 관련 서울 손...
3  우리 사회 그늘 자본 권력 예술 게릴라 천후 독립 예술 창작 집단 리슨 투 더시티 ...
4  아파트 그 공간 의미 우리나라 아파트 선호 나라 주택 해결 아파트 건설 우리 곳 아...
5  새해 시내 문화 예술 황금 띠 해 희망 찬 새해 시간 멀리 서울 시내 공연장 미술관...
6  서울시 문화 예술 프로그램 소개 서울시 향 예술의전당 세종 문화 회관 신년 음악회 ...
7  새해 볼 문화 행사 서울시 공연 전시 준비 웍스 박지윤 새해 가족 서울 공연장 미술...
(5, 390)
[[ 2.19999462  2.19999606  0.20000004 ... 10.19999455  4.19999551
   6.16115441]
 [ 0.20000215  0.20000157  0.20000097 ...  0.20000218  0.20000179
   0.20000525]
 [ 0.20000039  0.20000029  2.19537725 ...  0.2000004   0.20000033
   0.20000097]
 [ 0.2000007   0.20000051  0.20462077 ...  0.2000007   0.20000058
   0.23883411]
 [ 0.20000215  0.20000157  0.20000097 ...  0.20000218  0.20000179
   0.20000525]]


In [92]:
# lda_model = 벡터화시킨 텍스트 데이터를 fit까지만 적용한 모델
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print('\nTopic #', topic_idx+1)

        # topic 별로 1000개의 features 중에서 높은 값 순으로 정렬 후 index 반환
        #argsort = 디폴트가 오름차순 -> [::-1]로 내림차순으로 변경
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]

        # CountVectorizer 함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 features
        # 벡터화시킨 feature는 숫자-알파벳 순으로 정렬, 단어들 순서는 fit_transform 시키고 난 이후에도 동일
        feature_concat = '+'.join([str(feature_names[i])+'*'+str(round(topic[i], 1)) for i in top_idx])
        print(feature_concat)

feature_names = count_vect.get_feature_names()
display_topic_words(lda, feature_names, 5)


Topic # 1
문화*76.2+예술*38.7+서울시*35.2+공간*30.2+프로그램*29.2

Topic # 2
관객*0.2+예술가*0.2+부부*0.2+창작*0.2+친구*0.2

Topic # 3
아파트*31.2+작가*12.2+주택*12.1+시가지*10.2+상계*10.1

Topic # 4
더시티*17.0+리슨*17.0+예술*10.7+작품*7.9+한국*6.1

Topic # 5
관객*0.2+예술가*0.2+부부*0.2+창작*0.2+친구*0.2


In [93]:
doc_topics = lda.transform(ftr_vect)
print(doc_topics.shape)
print(doc_topics)

(8, 5)
[[9.97171827e-01 7.04595053e-04 7.09844429e-04 7.09138450e-04
  7.04595053e-04]
 [9.73971360e-04 9.66613309e-04 7.15351969e-01 2.81740833e-01
  9.66613309e-04]
 [9.97720806e-01 5.66945336e-04 5.71807629e-04 5.73496030e-04
  5.66945336e-04]
 [1.56979397e-01 1.47142293e-03 1.48502349e-03 8.38592734e-01
  1.47142293e-03]
 [8.64321432e-04 8.58599150e-04 9.96546804e-01 8.71675842e-04
  8.58599150e-04]
 [9.98536530e-01 3.63866082e-04 3.68220845e-04 3.67516580e-04
  3.63866082e-04]
 [9.94725335e-01 1.31634070e-03 1.32090086e-03 1.32108278e-03
  1.31634070e-03]
 [9.97399431e-01 6.47652158e-04 6.52894122e-04 6.52370986e-04
  6.47652158e-04]]


In [67]:
topic_names = ['Topic #' + str(i) for i in range(0,5)]
topic_df = pd.DataFrame(data=doc_topics, columns=topic_names)
print(topic_df)

   Topic #0  Topic #1  Topic #2  Topic #3  Topic #4
0  0.998493  0.000377  0.000375  0.000378  0.000377
1  0.000655  0.000652  0.000652  0.997378  0.000664
2  0.998888  0.000278  0.000277  0.000279  0.000278
3  0.001248  0.001248  0.001242  0.001252  0.995010
4  0.000619  0.142411  0.000614  0.855737  0.000620
5  0.999234  0.000192  0.000190  0.000192  0.000192
6  0.997309  0.000674  0.000671  0.000673  0.000673
7  0.998617  0.000345  0.000344  0.000346  0.000347
